In [89]:
# Dependencies
import pandas as pd
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
import re

# Step 1: Scraping

In [8]:
# Create a browser object for use by all the scrapes.
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

### Scrape https://mars.nasa.gov/news/ and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.


In [17]:
def scrape_mars_news():
    browser = init_browser()

    # Visit https://mars.nasa.gov/news/
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    # Give time for dynamic content to load
    time.sleep(10)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    #DEBUG    print(soup.prettify())
    #DEBUG    list(soup.children)

    # Get the title
    scrape_title = soup.find(class_='content_title')
    title = scrape_title.find_all('a')[0].text

    # Get the teaser
    scrape_teaser = soup.find('div', class_='article_teaser_body')
    teaser = scrape_teaser.get_text()

    # Store data in a dictionary
    mars_news = {
        "title": title,
        "teaser": teaser
    }

    # Close the browser after scraping
    browser.quit()

    # Return results
    return mars_news

In [18]:
mars_news = scrape_mars_news()
print(f'Title: {mars_news["title"]}')
print(f'Teaser: {mars_news["teaser"]}')

A Year of Surprising Science From NASA's InSight Mars Mission
A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet.


### Visit the url for JPL Featured Space Image at https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars. Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`. Make sure to find the image url to the full size `.jpg` image. Make sure to save a complete url string for this image.

In [376]:
def scrape_featured_image():
    browser = init_browser()

    # Format: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA00063_hires.jpg
    # Visit https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    base_url = "https://www.jpl.nasa.gov"
    browser.visit(url)

    # Give time for dynamic content to load
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Get the title
    scraped_url = soup.find('a', id='full_image')
    
    results = []

    for tmp in soup.find_all('a', {'class':'button fancybox'}):
        results.append(tmp['data-fancybox-href'])
        #DEBUG  print (results)

    featured_image_url = base_url + results[0] 
    
    # Close the browser after scraping
    browser.quit()

    # Return results
    return featured_image_url

In [377]:
featured_image_url = scrape_featured_image()
print(f'Featured Image URL: {featured_image_url}')

['/spaceimages/images/mediumsize/PIA17932_ip.jpg']
Featured Image URL: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17932_ip.jpg


### Visit the Mars Weather twitter account at https://twitter.com/marswxreport?lang=en and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [285]:
def scrape_tweet():
    browser = init_browser()

    #span.css-901oao.css-16my406.css-bfa6kz.r-1qd0xha.r-ad9z0x.r-bcqeeo.r-qvutc0
    
    #<span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight</span>

    # Visit https://twitter.com/marswxreport?lang=en
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)

    # Give time for dynamic content to load
    time.sleep(5)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Get the title
    #scraped_tweet = soup.find('span', class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
    #print(scraped_tweet)
    #results = []
    scraped_tweet = soup.find_all(text=True)
    #print(scraped_tweet)

    #for tmp in soup.find_all('a', {'class':'button fancybox'}):
    #    results.append(tmp['data-link'])
        #DEBUG print (results)
    
    tweet = ''
    for text in scraped_tweet:
        #print(f'text: {text}')
        if re.search(r'^InSight', text):
            #print(f'tweet: {text}')
            tweet = text
            break
        
    #tweet = scraped_tweet
        
    #x = scraped_tweet['class']
    #print(x)
    #txt = '.'.join(x)
    #txt = 'span' + '.' + txt
    #txt = x[0]
    #print(txt)

    #DEBUG        print(soup.prettify())
    #DEBUG    list(soup.children)
    
        # Visit https://twitter.com/marswxreport?lang=en
    #url = "https://twitter.com/marswxreport?lang=en"
    #browser.visit(url)

    # Give time for dynamic content to load
    #time.sleep(5)

    # Scrape page into Soup
    #html = browser.html
    #soup = bs(html, "html.parser")
    
    #new_scraped_tweet = soup.select_one(txt)


    #tweet = scraped_tweet.next_sibling
    #tweet = new_scraped_tweet.text
    #print(tweet)
    
    # Close the browser after scraping
    browser.quit()

    # Return results
    return tweet

In [286]:
mars_weather = scrape_tweet()
print(f'Tweet: {mars_weather}')

Tweet: InSight sol 443 (2020-02-24) low -94.8ºC (-138.6ºF) high -12.3ºC (9.8ºF)
winds from the SSE at 7.0 m/s (15.6 mph) gusting to 22.2 m/s (49.6 mph)
pressure at 6.30 hPa


### Visit the Mars Facts webpage at https://space-facts.com/mars/ and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc. Use Pandas to convert the data to a HTML table string.

In [296]:
# Visit https://space-facts.com/mars/ 
url = "https://space-facts.com/mars/"

# Read in tables.
tables = pd.read_html(url)

# Convert first table to dataframe.
df = tables[0]

# Store dataframe as an HTML table string.
html_table = df.to_html()

# Strip newline characters.
html_table = html_table.replace('\n', '')

In [297]:
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </

### Visit the USGS Astrogeology site at https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres. You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image. Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`. Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [364]:
browser = init_browser()

# Visit https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars 
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

# Give time for dynamic content to load
time.sleep(5)
    
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
hemispheres = soup.find_all('div', class_='description')
#print(hemispheres)

hemisphere_list = []
hemisphere_dict = {}
    
# Iterate through each hemisphere
for hemisphere in hemispheres:
    # Use Beautiful Soup's find() method to navigate and retrieve attributes
    #print(hemisphere)
    link = hemisphere.find('a')
    #print(f'LINK: {link}')
    title = link.find('h3').text.strip()
    #href = re.search(r'href="([^"]+)".*', link)
    href = re.search('\/(.*)\"', str(link)).group(1)
    link = 'https://astrogeology.usgs.gov/' + href
    #print(f'HREF: {href}')
    #print('-----------')
    #print(title)
    #http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
    #print(link)
    
    browser.visit(link)

    # Give time for dynamic content to load
    time.sleep(3)

    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')
    div = soup.find('div', class_='downloads')
    a = div.find('a')
    img_url = a['href']

    hemisphere_dict = {'title': title, 'img_url': img_url}
    hemisphere_list.append(hemisphere_dict)
        
    #print(f'IMAGE_LINK: {href}')
    
    # Click the 'Next' button on each page
    #try:
    #    browser.click_link_by_partial_text('next')
          
    #except:
    #    print("Scraping Complete")

# Close the browser after scraping
browser.quit()

In [365]:
hemisphere_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# Step 2: MongoDB and Flask Application

### Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above. Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

### Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function. Store the return value in Mongo as a Python dictionary.

### Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

### Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

# Step 3: Submission

### Notebook(s), screenshots of your final application, link to your new repository to BootCampSpot.